# Challenge Hands-On Session: Nuclear Masses

### Author(s): Alexandra Semposki, Rahul Jain

### 26 June 2023

Welcome to the challenge problems of the BMM session! You've made it to undiscovered country!

Now that you've gained a good level of understanding from the first hands-on notebook, we'll expand to look at the application of Bayesian Model Mixing (BMM) to nuclear masses. Let's first do an import of the masses data and see what it looks like.

In [ ]:
# required imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
from scipy.stats import norm
from scipy.special import gamma
import sys
import os

In [ ]:
# import Taweret and Latex if not already done in current codespace
#!pip install latex
# !git clone https://github.com/asemposki/Taweret.git 
# !cd Taweret && git checkout develop

# import from Taweret necessary attributes for part 1
from Taweret.Taweret.core.base_model import BaseModel
from Taweret.Taweret.core.base_mixer import BaseMixer
from Taweret.Taweret.mix.gaussian import Multivariate

In [ ]:
# define plot function for nuclear masses
def plot_nuclear(df,col):
    fig = plt.figure()
    ax = plt.axes()
    fig.patch.set_facecolor('white')
    fig.set_dpi(150)
    s = ax.scatter(df['N'], df['Z'], c=df[col],marker='s',label=col+('  (B.E. in MeV)'))
    ax.set_xlabel('N (neutrons)')
    ax.set_ylabel('Z (protons)')
    ax.legend()
    fig.colorbar(s, ax=ax)
    plt.show()

## Exploring nuclear mass data

In [ ]:
# import nuclear masses from csv file
df = pd.read_csv('BMM_Masses.csv')
display(df)

OK, great! So how do we interpret the data we just imported? 

Check out the column headers; the __Z__ and __N__ columns indicate the number of protons and neutrons, respectively, in a given nucleus. The four other categories are averages over sets of models that can be described by the types of methods they use---for example, __Skyrme__ uses the Skyrme energy density functional. 

If you're very curious about the forms of these models individually, check out the reference list below.

- Skyrme: Density Functional Theory models based on Skyrme Energy Density Functional
    - SkM* - https://www.sciencedirect.com/science/article/pii/0375947482904031?via%3Dihub
    - SkP - https://www.sciencedirect.com/science/article/pii/0375947484904330?via%3Dihub
    - SLy4 - https://iopscience.iop.org/article/10.1088/0031-8949/1995/T56/034
    - SV-min - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.79.034310


- UNEDF: Density Functional Theory models also based on Skyrme Energy Density Functional
    - UNEDF0 - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.82.024313
    - UNEDF1 - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.85.024304
    - UNEDF2 - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.89.054314


- Gogny: Density Functional Theory models based on Gogny Energy Density Functional
    - BCPM - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.87.064305
    - D1M - https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.102.242501


- Astro: More phenomenological mass models commonly used in nuclear astrophysics
    - FRDM2012 - https://www.sciencedirect.com/science/article/pii/S0092640X1600005X?via%3Dihub
    - HFB24 - https://journals.aps.org/prc/abstract/10.1103/PhysRevC.88.024308
    
    
Now we will plot one of the models over the nuclear chart, along with its uncertainties. These were assigned by averaging the individual model uncertainties within each of the groups of models above. The variable we're plotting in is the __binding energy__ in MeV (abbreviated B.E. in the plot legend). 

In [ ]:
plot_nuclear(df,'Skyrme')

In [ ]:
plot_nuclear(df,'Skyrme_sd')

__Question__: You can see that the binding energy uncertainties above are very small for the line passing diagonally through the center of the nuclear chart band. Can you guess why that might be happening?

---

# Challenge Project I: Multivariate BMM in 1-dimension: Investigating Isotopes

## Introduction

Now that we know how to pull in our data and plot it, we're curious about what it's telling us about the nuclei it pertains to. 

Say we are interested in understanding which models work best for a slice of the input space---in our case, we want to study one element and all of its modelled isotopes. We'll fix a proton number, $Z$, and pull the data sets for every nucleus that has that specific $Z$. Each of the four mass models we have access to in the dataframe above have predictions and uncertainties for all nuclei considered, so we can pick any of the possibilities from the dataframe. 

Let's start with lead (Pb, $Z$=82). __You can change this yourselves later and study other values of $Z$!__

In [ ]:
# pick Z = 82 and consider neutrons as the input space
df_82 = df.loc[df['Z'] == 82].copy()
display(df_82)

Now we can plot the model values vs. the neutron number in a one-dimensional input space sense.

In [ ]:
# plot the models 
fig, ax = plt.subplots(figsize=(8,6), dpi=150)
for i in ['Skyrme', 'UNEDF', 'Gogny', 'Astro']:
    ax.plot(df_82['N'], df_82[i], label=i)
    ax.fill_between(df_82['N'], df_82[i]-df_82[i+'_sd'], \
                     df_82[i]+df_82[i+'_sd'], alpha=0.2)
ax.set_xlabel('N')
ax.set_ylabel('Binding Energies [MeV]')
ax.set_title('Lead isotopes')
ax.legend(loc='lower right')
plt.show();

Let's look at how the curve appears from the four different models. 

In [ ]:
ax.set_xlim(140,max(df_82['N']))
ax.set_ylim(1675,1840)
fig

__Question__: The graph appears to have a wavy structure in the mean and uncertainties, regardless of the model used, across the values of $N$. Why do you think this is occurring?

---
---

### *** Challenge time!! ***

Now that you've got the data and have chosen an isotope to look at, try using BMM via Taweret's `Multivariate` class to mix the four models together and see what happens with the PPD and the weights. We outline a step-by-step process in the cells below to help you get started, but leave most of the coding up to you! Good luck!

__Some tips:__
- Your model wrapper class(es) must use all of the functions in the BaseModel class, since it is an abstract base class, even if they are not implemented, so make sure to define all of the models
- Taweret requires arrays when using Multivariate, so you'll need to convert your dataframe columns into numpy arrays using `to_numpy()`
- Taweret's BaseModel `evaluate` function requires you to send in the input space array even if it is not directly being used, so don't forget to make that an argument in your `evaluate` function
- Plot the individual models along with the PPD so you can tell which models are favoured in which region of the input space

In [ ]:
# step 1: write four (or one, depending on how creative you are) models that 
# Taweret will be able to use to obtain the mean and standard deviation 
# for the model across the number of neutrons

### YOUR CODE HERE ###

In [ ]:
# step 2: create a dict of the four models for Taweret

### YOUR CODE HERE ###

In [ ]:
# step 3: use Taweret to mix the four models through the `Multivariate` class

### YOUR CODE HERE ###

In [ ]:
# step 4: plot the PPD result and interpret it

### YOUR CODE HERE ###

__Question__: What does your PPD result look like? What model is considered the best in which region? Do the uncertainty bands look more reasonable than just using one of the models across the space? Compare with the individual models' uncertainty bands. 

In [ ]:
# step 5: plot the weights of each model using Taweret's `Multivariate`
# class functions and interpret them

### YOUR CODE HERE ###

__Question__: Do the weights make sense based on the result you obtained for the PPD?

---
## Concluding questions and exercises

1. What are some improvements you could make on the PPD you got from mixing the models you have currently? 
2. Try a different element other than lead. How different is the PPD for that element compared to the one you obtained for lead? How do the weights of each model change? Are other models favoured in regions where they were not for lead? Why could this be? 
3. If you're feeling up to it, look into the models we're using and why they might work better in certain regions of the nuclear chart compared to others.
4. If you're super curious, try fixing a neutron value instead of a proton value and seeing how the results of this project change.

---
---

Exercise written by: Alexandra Semposki

Nuclear masses provided by: Rahul Jain